질의 응답 시스템(QA System)
  - 사용자의 질문에 대한 적절한 답변을 반환하는 시스템
  - 추출형 QA(Extactive QA) : 문서에서 정답을 찾아서 반환(EX BERT기반 SQuAD모델)
  - 생성형 QA(Generative QA) : 정답을 직접 생성(EX GPT 기반 모델)


챗봇
  - 사용자와 자연스럽게 대화하는 프로그램
  - 롤 기반 챗봇 : 미리 정의된 규칙에 따라 답변하는 챗봇
  - 딥러닝 기반 챗봇 : 기계학습 모델이 문맥을 이해하고 대화하는 챗봇

Hugging Face의 transformer 라이브러리를 활용하여 사전학습된 BERT 모델을 사용한 질의 응답 시스템 구축

In [1]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

pipeline을 이용한 간단한 QA 모델

In [5]:
from transformers import pipeline
# 사전 학습된 BERT 기반 질의응답 모델 로드
qa_pipeline = pipeline("question-answering")

# 예제문서
context = '''
ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델입니다.
이 모델은 자연어 처리를 기반으로 질문에 답하거나 텍스트를 생성할 수 있습니다.
특히 GPT-4는 더 많은 파라메터를 가지고 있으며, 다양한 언어적 맥락을 이해하는데 강력합니다.
'''

# 질문
question = "ChatGPT는 무엇인가요"

# 답변 예측
# score 신뢰도
# start,end  : 정답이 문서에서 어디에 위치하는지
answer = qa_pipeline(question=question, context=context)
answer

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'score': 0.0605824813246727,
 'start': 39,
 'end': 90,
 'answer': '이 모델은 자연어 처리를 기반으로 질문에 답하거나 텍스트를 생성할 수 있습니다.\n특히 GPT'}

BERT 모델을 직접 활용한 질의 응답 시스템
  - Hugging Face의 AutoModelForQuestionAnswering 을 사용해서 BERT 기반의 질의응답 시스템을 직접 구현

In [27]:
import torch.nn.functional as F
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# 모델과 토크나이져 로드
model_name = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# 예제문서
context = '''
ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델입니다.
이 모델은 자연어 처리를 기반으로 질문에 답하거나 텍스트를 생성할 수 있습니다.
특히 GPT-4는 더 많은 파라메터를 가지고 있으며, 다양한 언어적 맥락을 이해하는데 강력합니다.
'''

# 질문
question = "ChatGPT는 무엇인가요"

# 토큰화
inputs = tokenizer(question,context,return_tensors='pt', max_length=512, truncation=True)

# 모델 예측
with torch.no_grad():
  outputs = model(**inputs)

# 확률값 계산
start_probs = F.softmax(outputs.start_logits, dim=1)
end_probs = F.softmax(outputs.end_logits, dim=1)

# 정답 인덱스 찾기
answer_start = torch.argmax(start_probs)  # 정답 시작 인덱스
answer_end = torch.argmax(end_probs) + 1  # 정답 끝 인덱스

# 정답 추출
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end])
)


answer

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'openai에서 개발한 대화형 인공지능 모델입니다'

딥러닝 기반 쳇봇